# Hyperpamaters Tuning

In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import pandas as pd
import random

from sklearn.preprocessing import MinMaxScaler
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, TimeDistributed, RepeatVector
from keras.optimizers import Adam

import keras_tuner

from lib.read_data import read_and_join_output_file
#from lib.create_pipeline import create_transformation_pipeline
from lib.deeplearning import create_transformation_pipelines, evaluate_forecast, reshape_data_to_3d, get_sets_shapes
from lib.transform_impute import convert_back_df
from lib.split_data import train_test_group_time_split

In [3]:
RANDOM_SEED = 31
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tensorflow.random.set_seed(RANDOM_SEED)

In [4]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## Preparing the Dataset
The train and test sets are split by Township-Ranges, i.e. some Township-Ranges data are either fully in the train or test set.
The target value is the value of that variable for 2021
Thus train/test sets are of shape (number of Township-Ranges, 7 years (2014-2020), the number of features).
The input of 1 data point in the model is of shape (7x81


In [5]:
test_size=0.15
# Load the data from the ETL output files
X = read_and_join_output_file()
# Split the data into a training and a test set
X_train_df, X_test_df, y_train_df, y_test_df = train_test_group_time_split(X, index=["TOWNSHIP_RANGE", "YEAR"], group="TOWNSHIP_RANGE", test_size=test_size, random_seed=RANDOM_SEED)
# Create, fit and apply the data imputation pipeline to the training and test sets
impute_pipeline, columns = create_transformation_pipelines(X_train_df)
X_train_impute = impute_pipeline.fit_transform(X_train_df)
X_test_impute = impute_pipeline.transform(X_test_df)
# Convert the X_train and X_test back to dataframes
X_train_impute_df = pd.DataFrame(X_train_impute, index=X_train_df.index, columns=columns)
X_test_impute_df = pd.DataFrame(X_test_impute, index=X_test_df.index, columns=columns)
# Keep only the GSE_GWE variable as the outcome variable
scaler = MinMaxScaler()
y_train = scaler.fit_transform(y_train_df[["GSE_GWE"]])
y_train_3d = y_train[..., np.newaxis]
y_test = scaler.transform(y_test_df[["GSE_GWE"]])
nb_features = len(X_train_impute_df.columns)
X_train_impute_df

TOTALDRILLDEPTH_AVG  WELLYIELD_AVG  STATICWATERLEVEL_AVG  \
TOWNSHIP_RANGE YEAR                                                             
T01N R02E      2014             0.000000       0.000489              0.020868   
               2015             0.000000       0.000000              0.000000   
               2016             0.000000       0.003259              0.036728   
               2017             0.066667       0.006410              0.025876   
               2018             0.053651       0.000652              0.063022   
...                                  ...            ...                   ...   
T32S R30E      2016             0.000000       0.000000              0.000000   
               2017             0.000000       0.000000              0.000000   
               2018             0.000000       0.000000              0.000000   
               2019             0.000000       0.000000              0.000000   
               2020             0.000000       0.000000              0.000000   

                     TOPOFPERFORATEDINTERVAL_AVG  \
TOWNSHIP_RANGE YEAR                                
T01N R02E      2014                     0.052288   
               2015                     0.000000   
               2016                     0.084967   
               2017                     0.082789   
               2018                     0.077124   
...                                          ...   
T32S R30E      2016                     0.000000   
               2017                     0.000000   
               2018                     0.000000   
               2019                     0.000000   
               2020                     0.000000   

                     BOTTOMOFPERFORATEDINTERVAL_AVG  TOTALCOMPLETEDDEPTH_AVG  \
TOWNSHIP_RANGE YEAR                                                            
T01N R02E      2014                        0.076699                 0.127841   
               2015                        0.000000                 0.000000   
               2016                        0.058252                 0.056818   
               2017                        0.064725                 0.074495   
               2018                        0.053592                 0.064015   
...                                             ...                      ...   
T32S R30E      2016                        0.000000                 0.000000   
               2017                        0.000000                 0.000000   
               2018                        0.000000                 0.000000   
               2019                        0.000000                 0.000000   
               2020                        0.000000                 0.000000   

                     VEGETATION_BLUE_OAK-GRAY_PINE  \
TOWNSHIP_RANGE YEAR                                  
T01N R02E      2014                       0.010798   
               2015                       0.010798   
               2016                       0.010798   
               2017                       0.010798   
               2018                       0.010798   
...                                            ...   
T32S R30E      2016                       0.033178   
               2017                       0.033178   
               2018                       0.033178   
               2019                       0.033178   
               2020                       0.033178   

                     VEGETATION_CALIFORNIA_COAST_LIVE_OAK  \
TOWNSHIP_RANGE YEAR                                         
T01N R02E      2014                              0.002749   
               2015                              0.002749   
               2016                              0.002749   
               2017                              0.002749   
               2018                              0.002749   
...                                                   ...   
T32S R30E      2016                              0.000000 

In [6]:
X_train, X_test = reshape_data_to_3d([X_train_impute_df, X_test_impute_df])
get_sets_shapes(X_train, X_test)

,nb_items,nb_timestamps,nb_features
training dataset,406,7,81
test dataset,72,7,81


## Simple Model Hyper-parameter Tuning
This model is just made of a single LSTM layer

In [6]:
class Model1(keras_tuner.HyperModel):
    def build(self, hp):
        model = Sequential()
        hp_units = hp.Int("units", min_value=10, max_value=300, step=10)
        hp_activ = hp.Choice("activation", values=["tanh", "sigmoid"])
        model.add(LSTM(units=hp_units, activation=hp_activ, input_shape=(7, nb_features)))
        model.add(Dense(1, activation="linear"))
        hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        model.compile(loss="mse", optimizer=Adam(learning_rate=hp_learning_rate), metrics=[keras.metrics.RootMeanSquaredError()])
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            validation_split=hp.Choice("validation_split", values=[0.05, 0.1, 0.15, 0.2]),
            batch_size=hp.Int("batch_size", min_value=32, max_value=192, step=16),
            epochs=hp.Int("epochs", min_value=30, max_value=500, step=5),
            shuffle=True,
            **kwargs,
        )

In [7]:
stop_early = tensorflow.keras.callbacks.EarlyStopping(monitor='val_root_mean_squared_error', patience=10, verbose=1)
tuner = keras_tuner.BayesianOptimization(Model1(),
                             objective=keras_tuner.Objective("val_root_mean_squared_error", direction="min"),
                             max_trials=250,
                             beta=3.2,
                             seed=RANDOM_SEED,
                             overwrite=True,
                             directory="keras_tuner",
                             project_name="model1_tuner")
tuner.search(X_train, y_train, callbacks=[stop_early])

Trial 250 Complete [00h 00m 05s]
val_root_mean_squared_error: 0.09763942658901215

Best val_root_mean_squared_error So Far: 0.0824510008096695
Total elapsed time: 01h 05m 47s
INFO:tensorflow:Oracle triggered exit


### Best Model Hyperparameters

In [8]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The hyperparameter search is complete.
validation_split: {best_hps.get('validation_split')}
lstm_units: {best_hps.get('units')}
lstm_activation: {best_hps.get('activation')}
learning_rate: {best_hps.get('learning_rate')}
batch_size: {best_hps.get('batch_size')}
epochs: {best_hps.get('epochs')}
""")


The hyperparameter search is complete.
validation_split: 0.15
lstm_units: 60
lstm_activation: tanh
learning_rate: 0.01
batch_size: 112
epochs: 30



### Hyperparameters Tuning Summary

In [9]:
tuner.results_summary()

Results summary
Results in keras_tuner\model1_tuner
Showing 10 best trials
Trial summary
Hyperparameters:
units: 60
activation: tanh
learning_rate: 0.01
validation_split: 0.15
batch_size: 112
epochs: 30
Score: 0.0824510008096695
Trial summary
Hyperparameters:
units: 60
activation: tanh
learning_rate: 0.01
validation_split: 0.15
batch_size: 112
epochs: 30
Score: 0.08390998095273972
Trial summary
Hyperparameters:
units: 60
activation: tanh
learning_rate: 0.01
validation_split: 0.15
batch_size: 112
epochs: 30
Score: 0.08468343317508698
Trial summary
Hyperparameters:
units: 60
activation: tanh
learning_rate: 0.01
validation_split: 0.15
batch_size: 112
epochs: 30
Score: 0.08546236157417297
Trial summary
Hyperparameters:
units: 60
activation: tanh
learning_rate: 0.01
validation_split: 0.15
batch_size: 112
epochs: 30
Score: 0.08679324388504028
Trial summary
Hyperparameters:
units: 60
activation: tanh
learning_rate: 0.01
validation_split: 0.15
batch_size: 112
epochs: 30
Score: 0.08681566268205

## Model2 Hyper-parameter tuning
This model is made of
* a simple or bidirectional LSTM layer
* a Dense unit
* a Dropout Unit

In [10]:
class Model2(keras_tuner.HyperModel):
    def build(self, hp):
        model = Sequential()
        lstm_units = hp.Int("lstm_units", min_value=10, max_value=300, step=10)
        lstm_activ = hp.Choice("lstm_activation", values=["tanh", "sigmoid"])
        model.add(LSTM(units=lstm_units, activation=lstm_activ, input_shape=(7, nb_features)))
        dense_units = hp.Int("dense_units", min_value=11, max_value=101, step=2)
        dense_activation = hp.Choice("dense_activation", values=["relu", "tanh", "sigmoid"])
        model.add(Dense(dense_units, activation=dense_activation))
        hp_dropout = hp.Float("dropout_rate", min_value=0.05, max_value=0.25, step=0.05)
        model.add(Dropout(hp_dropout))
        model.add(Dense(1, activation="linear"))
        hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        model.compile(loss="mse", optimizer=Adam(learning_rate=hp_learning_rate), metrics=[keras.metrics.RootMeanSquaredError()])
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            validation_split=hp.Choice("validation_split", values=[0.05, 0.1, 0.15, 0.2]),
            batch_size=hp.Int("batch_size", min_value=32, max_value=192, step=16),
            epochs=hp.Int("epochs", min_value=30, max_value=500, step=5),
            shuffle=True,
            **kwargs,
        )

In [11]:
stop_early = tensorflow.keras.callbacks.EarlyStopping(monitor="val_root_mean_squared_error", patience=10, verbose=1)
tuner = keras_tuner.BayesianOptimization(Model2(),
                              objective=keras_tuner.Objective("val_root_mean_squared_error", direction="min"),
                              max_trials=400,
                              beta=3.2,
                              seed=RANDOM_SEED,
                              overwrite=True,
                              directory="keras_tuner",
                              project_name="model2_tuner")
tuner.search(X_train, y_train, callbacks=[stop_early])

Trial 400 Complete [00h 00m 06s]
val_root_mean_squared_error: 0.08002614974975586

Best val_root_mean_squared_error So Far: 0.06468775868415833
Total elapsed time: 03h 14m 13s
INFO:tensorflow:Oracle triggered exit


### Best Model Hyperparameters

In [12]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The hyperparameter search is complete.
validation_split: {best_hps.get('validation_split')}
lstm_units: {best_hps.get('lstm_units')}
lstm_activation: {best_hps.get('lstm_activation')}
dense_units: {best_hps.get('dense_units')}
dense_activation: {best_hps.get('dense_activation')}
dropout_rate: {best_hps.get('dropout_rate')}
learning_rate: {best_hps.get('learning_rate')}
batch_size: {best_hps.get('batch_size')}
epochs: {best_hps.get('epochs')}
""")


The hyperparameter search is complete.
validation_split: 0.05
lstm_units: 10
lstm_activation: sigmoid
dense_units: 11
dense_activation: relu
dropout_rate: 0.05
learning_rate: 0.01
batch_size: 32
epochs: 500



### Hyperparameters Tuning Summary

In [13]:
tuner.results_summary()

Results summary
Results in keras_tuner\model2_tuner
Showing 10 best trials
Trial summary
Hyperparameters:
lstm_units: 10
lstm_activation: sigmoid
dense_units: 11
dense_activation: relu
dropout_rate: 0.05
learning_rate: 0.01
validation_split: 0.05
batch_size: 32
epochs: 500
Score: 0.06468775868415833
Trial summary
Hyperparameters:
lstm_units: 10
lstm_activation: sigmoid
dense_units: 11
dense_activation: relu
dropout_rate: 0.1
learning_rate: 0.01
validation_split: 0.05
batch_size: 32
epochs: 500
Score: 0.06481722742319107
Trial summary
Hyperparameters:
lstm_units: 10
lstm_activation: sigmoid
dense_units: 11
dense_activation: relu
dropout_rate: 0.15000000000000002
learning_rate: 0.01
validation_split: 0.05
batch_size: 32
epochs: 415
Score: 0.06549284607172012
Trial summary
Hyperparameters:
lstm_units: 10
lstm_activation: sigmoid
dense_units: 11
dense_activation: relu
dropout_rate: 0.15000000000000002
learning_rate: 0.01
validation_split: 0.05
batch_size: 32
epochs: 415
Score: 0.0660640969

## Model3 Hyper-parameter tuning
* a simple or bidirectional encoding LSTM layer
* a simple or bidirectional decoding LSTM layer
* a Dense unit
* a Dropout Unit

In [7]:
class Model3(keras_tuner.HyperModel):
    def build(self, hp):
        model = Sequential()
        lstm_units = hp.Int("lstm_units", min_value=10, max_value=300, step=10)
        lstm_activ = hp.Choice("lstm_activation", values=["tanh", "sigmoid"])
        model.add(LSTM(units=lstm_units, activation="sigmoid", input_shape=(7, nb_features)))
        model.add(RepeatVector(1))
        lstm_units_2 = hp.Int("2nd_lstm_units", min_value=10, max_value=300, step=10)
        lstm_activ_2 = hp.Choice("2nd_lstm_activation", values=["tanh", "sigmoid"])
        model.add(LSTM(units=lstm_units_2, activation="sigmoid", return_sequences=True))
        dense_units = hp.Int("dense_units", min_value=11, max_value=101, step=2)
        dense_activation = hp.Choice("dense_activation", values=["relu", "tanh", "sigmoid"])
        model.add(TimeDistributed(Dense(dense_units, activation=dense_activation)))
        hp_dropout = hp.Float("dropout_rate", min_value=0.05, max_value=0.25, step=0.05)
        model.add(Dropout(hp_dropout))
        model.add(Dense(1, activation="linear"))
        hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        model.compile(loss="mse", optimizer=Adam(learning_rate=hp_learning_rate), metrics=[keras.metrics.RootMeanSquaredError()])
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            validation_split=hp.Choice("validation_split", values=[0.05, 0.1, 0.15, 0.2]),
            batch_size=hp.Int("batch_size", min_value=32, max_value=192, step=16),
            epochs=hp.Int("epochs", min_value=30, max_value=500, step=5),
            shuffle=True,
            **kwargs,
        )

In [8]:
stop_early = tensorflow.keras.callbacks.EarlyStopping(monitor="val_root_mean_squared_error", patience=10, verbose=1)
tuner = keras_tuner.BayesianOptimization(Model3(),
                              objective=keras_tuner.Objective("val_root_mean_squared_error", direction="min"),
                              max_trials=400,
                              beta=3.2,
                              seed=RANDOM_SEED,
                              overwrite=True,
                              directory="keras_tuner",
                              project_name="model3_tuner")
tuner.search(X_train, y_train, callbacks=[stop_early])

Trial 500 Complete [00h 00m 18s]
val_root_mean_squared_error: 0.1150362491607666

Best val_root_mean_squared_error So Far: 0.06309118866920471
Total elapsed time: 07h 29m 36s
INFO:tensorflow:Oracle triggered exit


### Best Model Hyperparameters

In [9]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The hyperparameter search is complete.
validation_split: {best_hps.get('validation_split')}
lstm_units: {best_hps.get('lstm_units')}
lstm_activation: {best_hps.get('lstm_activation')}
2nd_lstm_units: {best_hps.get('2nd_lstm_units')}
2nd_lstm_activation: {best_hps.get('2nd_lstm_activation')}
dense_units: {best_hps.get('dense_units')}
dense_activation: {best_hps.get('dense_activation')}
dropout_rate: {best_hps.get('dropout_rate')}
learning_rate: {best_hps.get('learning_rate')}
batch_size: {best_hps.get('batch_size')}
epochs: {best_hps.get('epochs')}
""")


The hyperparameter search is complete.
validation_split: 0.05
lstm_units: 300
lstm_activation: sigmoid
2nd_lstm_units: 300
2nd_lstm_activation: tanh
dense_units: 11
dense_activation: sigmoid
dropout_rate: 0.05
learning_rate: 0.01
batch_size: 32
epochs: 30



### Hyperparameters Tuning Summary

In [10]:
tuner.results_summary()

Results summary
Results in keras_tuner\model3_tuner
Showing 10 best trials
Trial summary
Hyperparameters:
lstm_units: 300
lstm_activation: sigmoid
2nd_lstm_units: 300
2nd_lstm_activation: tanh
dense_units: 11
dense_activation: sigmoid
dropout_rate: 0.05
learning_rate: 0.01
validation_split: 0.05
batch_size: 32
epochs: 30
Score: 0.06309118866920471
Trial summary
Hyperparameters:
lstm_units: 300
lstm_activation: sigmoid
2nd_lstm_units: 300
2nd_lstm_activation: tanh
dense_units: 11
dense_activation: sigmoid
dropout_rate: 0.05
learning_rate: 0.01
validation_split: 0.05
batch_size: 32
epochs: 30
Score: 0.06499446928501129
Trial summary
Hyperparameters:
lstm_units: 300
lstm_activation: sigmoid
2nd_lstm_units: 300
2nd_lstm_activation: sigmoid
dense_units: 11
dense_activation: sigmoid
dropout_rate: 0.05
learning_rate: 0.01
validation_split: 0.05
batch_size: 32
epochs: 30
Score: 0.06594016402959824
Trial summary
Hyperparameters:
lstm_units: 300
lstm_activation: sigmoid
2nd_lstm_units: 300
2nd_l